# Reanding Exif data from ./images

In [ ]:
import os
from PIL import Image

dir = r"images/"
themes = os.listdir(dir)
data = []

for theme in themes: 
    if theme in {".DS_Store", "ExifData.json"}:
        pass   
    else:
        print(theme)
        for image in os.listdir(dir + theme):
            #print(image)
            if image.endswith(".jpg") :
                img_path =  dir + theme + '/'+ image
                # print(img_path)
                with Image.open(img_path) as img:
                    # print(img)
                    width, height = img.size
                    format = img.format
                    try:
                        orientation = img._getexif()[274]
                    except:
                        orientation = "no exif"
                    #print(f'{width=}x{height=} {format=} {orientation=}')
                    data.append({"theme": theme, "image": image, "width": width, "height": height, "format": format, "orientation": orientation})
                    
print(data[0:5])

## Get proiminant colors from images

In [ ]:
from PIL import Image
import warnings
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import MiniBatchKMeans

def getKmeans(img):
    # init
    img = Image.open(img) # open image
    imgfile = img.resize((int(img.size[0]/5), int(img.size[1]/5))) # resize
    numarray = numpy.array(imgfile.getdata(), numpy.uint8) # convert image to numpy array
    cluster_count = 5 # number of clusters
    clusters = MiniBatchKMeans(n_clusters=cluster_count, n_init="auto") # init kmeans
    clusters.fit(numarray) # fit kmeans
    npbins = numpy.arange(0, cluster_count + 1) # init bins
    histogram = numpy.histogram(clusters.labels_, bins=npbins) # get histogram
    #labels = numpy.unique(clusters.labels_) # get labels

    # getting colors
    color = []
    for i in range(cluster_count):
        color.append(
            "#%02x%02x%02x" % (math.ceil(clusters.cluster_centers_[i][0]),math.ceil(clusters.cluster_centers_[i][1]),math.ceil(clusters.cluster_centers_[i][2]),)
        )
    #barlist = plot.bar(labels, sorted(histogram[0], reverse=True), color=color)
    #plot.show()
    mapping = {"numbers": sorted(histogram[0], reverse=True), "colors": color}
    return (mapping)

getKmeans("./images/Exoplanet/2M1207_b.jpg")

## Getting histogram from images

In [ ]:
from PIL import Image
import matplotlib.pyplot as plot
import numpy as np


def getDiagram(img):
    imgfile = Image.open("./images/Exoplanet/2M1207_b.jpg")
    histogram = imgfile.histogram()
    red = histogram[0:255]
    green = histogram[256:511]
    blue = histogram[512:767]
    # x = range(255)
    # y = []
    # for i in x:
    #     y.append((red[i], green[i], blue[i]))
    # figure,axes = plot.subplots()
    # axes.set_prop_cycle("color", ["red", "green", "blue"])
    # plot.plot(x, y)
    # plot.show()
    mapping = {"red": np.mean(red), "green": np.mean(green), "blue": np.mean(blue)}
    return mapping

getDiagram("./images/Exoplanet/2M1207_b.jpg")

## Adding diagram and Kmeans to exif data

In [ ]:
for obj in data:
    print(obj["image"])
    obj["kmeans"] = getKmeans(dir + obj["theme"] + '/' + obj["image"])
    obj["diagram"] = getDiagram(dir + obj["theme"] + '/' + obj["image"])

## Writing exif data into json file : ./images/ExifData.json

In [ ]:
import json
from numpyencoder import NumpyEncoder

with open('./images/ExifData2.json', 'w') as f:
    json.dump(data, f, indent=4, cls=NumpyEncoder)